**Import packages**

In [7]:
import sys
from pathlib import Path

# Get the parent directory of the current notebook
notebook_dir = Path.cwd()  # Folder containing the notebook
project_root = notebook_dir.parent  # Go up one level to 'project/'

# Add the 'src/' folder to Python's search path
sys.path.append(str(project_root / "src"))
sys.path.append(str(project_root / "data"))

# Verify
print(sys.path)
import yfinance as yf
from fredapi import Fred
import pandas as pd 
from dotenv import load_dotenv
import os
from ecbdata import ecbdata
import prophet as Prophet
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error

import numpy as np
from sklearn.preprocessing import StandardScaler
from prophet import Prophet
import plotly as plt 
import itertools


from dask.distributed import Client
from prophet.diagnostics import cross_validation , performance_metrics

['C:\\Users\\Radhouen\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\Radhouen\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\Radhouen\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\Radhouen\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\venv', '', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\venv\\lib\\site-packages', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\venv\\lib\\site-packages\\win32', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\venv\\lib\\site-packages\\Pythonwin', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\src', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\data', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\src', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\data', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\src', 'c:\\Users\\Radhouen\\Gold_Diggers\\gold-team\\dat

**Load Dataframe**

In [8]:
df=pd.read_csv('../notebooks/df_not_stationary.csv')
df=df.set_index(df['Date'])
df.head(30)

,Date,Gold,USD_Index,EUR_USD,SP_500,Oil,Bitcoin,Vix,CPI,Unemployment_Rate,...,Zinsen,Gold_ETFs,M2_Euro,M2_US,GPRD,DOW,MOY,Stunde_des_Tages,Jahr,Volles_Datum
Date,,,,,,,,,,,,,,,,,,,,,
2014-09-17,2014-09-17,1234.400024,84.699997,1.295908,2001.569946,94.419998,457.334015,12.650000,237.712067,5.793333,...,0.09,17.140000,9.487253e+12,11543.060000,159.797913,2,9,0,2014,2014-09-17
2014-09-18,2014-09-18,1225.699951,84.320000,1.285000,2011.359985,93.070000,424.440002,12.030000,237.692133,5.786667,...,0.09,15.500000,9.488013e+12,11545.520000,118.057968,3,9,0,2014,2014-09-18
2014-09-19,2014-09-19,1215.300049,84.800003,1.292006,2010.400024,92.410004,394.795990,12.110000,237.672200,5.780000,...,0.09,16.900000,9.488774e+12,11547.980000,132.806259,4,9,0,2014,2014-09-19
2014-09-20,2014-09-20,1215.300049,84.800003,1.292006,2010.400024,92.410004,408.903992,12.110000,237.652267,5.773333,...,0.09,16.746667,9.489535e+12,11550.440000,76.326447,5,9,0,2014,2014-09-20
2014-09-21,2014-09-21,1215.300049,84.800003,1.292006,2010.400024,92.410004,398.821014,12.110000,237.632333,5.766667,...,0.09,16.593333,9.490296e+12,11552.900000,79.820908,6,9,0,2014,2014-09-21
2014-09-22,2014-09-22,1216.800049,84.669998,1.284142,1994.290039,91.519997,402.152008,13.690000,237.612400,5.760000,...,0.09,16.440000,9.491056e+12,11555.360000,80.678589,0,9,0,2014,2014-09-22
2014-09-23,2014-09-23,1221.000000,84.660004,1.285149,1982.770020,91.559998,435.790985,14.930000,237.592467,5.753333,...,0.09,16.650000,9.491817e+12,11557.820000,179.175827,1,9,0,2014,2014-09-23
2014-09-24,2014-09-24,1218.599976,85.050003,1.285397,1998.300049,92.800003,423.204987,13.270000,237.572533,5.746667,...,0.09,17.210000,9.492578e+12,11560.280000,146.881638,2,9,0,2014,2014-09-24
2014-09-25,2014-09-25,1221.199951,85.199997,1.277808,1965.989990,92.529999,411.574005,15.640000,237.552600,5.740000,...,0.09,17.990000,9.493339e+12,11562.740000,107.528381,3,9,0,2014,2014-09-25


**Preparing the dataframe for Prophet**

In [9]:
gold_df=df['Gold'].reset_index()
gold_df=gold_df.rename(columns={'Gold': 'y','Date':'ds'})
gold_df

,ds,y
0,2014-09-17,1234.400024
1,2014-09-18,1225.699951
2,2014-09-19,1215.300049
3,2014-09-20,1215.300049
4,2014-09-21,1215.300049
...,...,...
3911,2025-06-02,3370.600098
3912,2025-06-03,3350.199951
3913,2025-06-04,3373.500000
3914,2025-06-05,3350.699951


In [11]:

gold_df.fillna(method='ffill', inplace=True)
gold_df.fillna(method='bfill', inplace=True)

C:\Users\Radhouen\AppData\Local\Temp\ipykernel_7080\476629285.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gold_df.fillna(method='ffill', inplace=True)
C:\Users\Radhouen\AppData\Local\Temp\ipykernel_7080\476629285.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  gold_df.fillna(method='bfill', inplace=True)


**Data Splitting**

In [10]:
train = gold_df[(gold_df['ds'] >= '2024-01-01') & (gold_df['ds']<'2025-05-01')]
test = gold_df[gold_df['ds'] >= '2025-05-01']
train

,ds,y
3393,2024-01-01,2062.399902
3394,2024-01-02,2064.399902
3395,2024-01-03,2034.199951
3396,2024-01-04,2042.300049
3397,2024-01-05,2042.400024
...,...,...
3874,2025-04-26,3282.399902
3875,2025-04-27,3282.399902
3876,2025-04-28,3332.500000
3877,2025-04-29,3318.800049


**Cross-Validation and finding the best parameters**

In [13]:



client = Client()  # connect to the cluster
cutoffs = pd.to_datetime(['2024-02-15', '2024-08-15','2025-02-15'])


param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.3, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 5, 7, 10.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params) 
    m.fit(train) # Fit model with given params
    df_cv = cross_validation(m, initial='100 days', horizon='30 days', parallel="dask")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

c:\Users\Radhouen\Gold_Diggers\gold-team\venv\lib\site-packages\distributed\node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 65427 instead
  warnings.warn(
13:08:21 - cmdstanpy - INFO - Chain [1] start processing
13:08:21 - cmdstanpy - INFO - Chain [1] done processing
13:08:26 - cmdstanpy - INFO - Chain [1] start processing
13:08:26 - cmdstanpy - INFO - Chain [1] done processing
13:08:28 - cmdstanpy - INFO - Chain [1] start processing
13:08:28 - cmdstanpy - INFO - Chain [1] done processing
13:08:30 - cmdstanpy - INFO - Chain [1] start processing
13:08:30 - cmdstanpy - INFO - Chain [1] done processing
13:08:33 - cmdstanpy - INFO - Chain [1] start processing
13:08:33 - cmdstanpy - INFO - Chain [1] done processing
13:08:35 - cmdstanpy - INFO - Chain [1] start processing
13:08:35 - cmdstanpy - INFO - Chain [1] done processing
13:08:37 - cmdstanpy - INFO - Chain [1] start processing
13:08:37 - cmdstanpy - INFO

    changepoint_prior_scale  seasonality_prior_scale        rmse
0                     0.001                     0.01   97.253918
1                     0.001                     0.10   98.659918
2                     0.001                     1.00   97.452924
3                     0.001                     5.00   98.154253
4                     0.001                     7.00   97.300262
5                     0.001                    10.00   96.616350
6                     0.010                     0.01  114.539357
7                     0.010                     0.10  114.135430
8                     0.010                     1.00  113.639505
9                     0.010                     5.00  113.566063
10                    0.010                     7.00  112.834309
11                    0.010                    10.00  112.749727
12                    0.100                     0.01  103.454185
13                    0.100                     0.10  103.443549
14                    0.1

**Best parameters Dataframe**

In [14]:
tuning_results.sort_values(by='rmse')

,changepoint_prior_scale,seasonality_prior_scale,rmse
5,0.001,10.00,96.616350
0,0.001,0.01,97.253918
4,0.001,7.00,97.300262
2,0.001,1.00,97.452924
3,0.001,5.00,98.154253
1,0.001,0.10,98.659918
24,0.500,0.01,102.510262
27,0.500,5.00,102.515417
28,0.500,7.00,102.523991
26,0.500,1.00,102.532363


**Training the model with the best parameters**

In [15]:
model = Prophet(
    changepoint_prior_scale=0.001,  
    seasonality_prior_scale=10,  
    n_changepoints=50,
    seasonality_mode='multiplicative',
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    growth='linear'
)


model.fit(train.rename(columns={'y': 'y'}))

13:14:28 - cmdstanpy - INFO - Chain [1] start processing
13:14:29 - cmdstanpy - INFO - Chain [1] done processing
13:14:29 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
Optimization terminated abnormally. Falling back to Newton.
13:14:29 - cmdstanpy - INFO - Chain [1] start processing
13:14:31 - cmdstanpy - INFO - Chain [1] done processing


**Prediction**

In [16]:
import plotly_express as px 
test_future = test.copy()
forecast = model.predict(test_future)
forecast


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2025-05-01,3058.327723,3193.552179,3302.143252,3058.327723,3058.327723,0.061414,0.061414,0.061414,0.001590,0.001590,0.001590,0.059824,0.059824,0.059824,0.0,0.0,0.0,3246.151852
1,2025-05-02,3060.527672,3185.590815,3291.861873,3060.527672,3060.527672,0.057343,0.057343,0.057343,0.000989,0.000989,0.000989,0.056353,0.056353,0.056353,0.0,0.0,0.0,3236.026858
2,2025-05-03,3062.727620,3167.779947,3280.933232,3062.727620,3062.727620,0.052642,0.052642,0.052642,-0.000120,-0.000120,-0.000120,0.052762,0.052762,0.052762,0.0,0.0,0.0,3223.955666
3,2025-05-04,3064.927569,3157.523199,3267.685527,3064.927569,3064.927569,0.047899,0.047899,0.047899,-0.001216,-0.001216,-0.001216,0.049115,0.049115,0.049115,0.0,0.0,0.0,3211.735227
4,2025-05-05,3067.127518,3149.994267,3253.088406,3067.127518,3067.127518,0.044055,0.044055,0.044055,-0.001423,-0.001423,-0.001423,0.045478,0.045478,0.045478,0.0,0.0,0.0,3202.250893
5,2025-05-06,3069.327467,3145.749345,3250.975593,3069.327466,3069.327467,0.041886,0.041886,0.041886,-0.000029,-0.000029,-0.000029,0.041916,0.041916,0.041916,0.0,0.0,0.0,3197.890438
6,2025-05-07,3071.527415,3140.403262,3249.004071,3071.527415,3071.527416,0.038697,0.038697,0.038697,0.000209,0.000209,0.000209,0.038488,0.038488,0.038488,0.0,0.0,0.0,3190.385903
7,2025-05-08,3073.727364,3133.626414,3240.572065,3073.727364,3073.727364,0.036842,0.036842,0.036842,0.001590,0.001590,0.001590,0.035252,0.035252,0.035252,0.0,0.0,0.0,3186.970955
8,2025-05-09,3075.927313,3125.520011,3232.050649,3075.927312,3075.927313,0.033248,0.033248,0.033248,0.000989,0.000989,0.000989,0.032258,0.032258,0.032258,0.0,0.0,0.0,3178.194509
9,2025-05-10,3078.127262,3114.122017,3222.445704,3078.127261,3078.127262,0.029429,0.029429,0.029429,-0.000120,-0.000120,-0.000120,0.029549,0.029549,0.029549,0.0,0.0,0.0,3168.712484


**Visualization of the prediction and actual value of the price of gold for the last week of the test dataframe**

In [17]:
import plotly_express as px
viz_df = test.copy()
forecasted_values = forecast.tail(len(test))['yhat']
viz_df['yhat'] = forecasted_values.values
px.line(viz_df,viz_df.ds,[viz_df.y,viz_df.yhat])

**RMSE calculated the last week**

In [18]:
root_mean_squared_error(viz_df.y, viz_df.yhat)

130.58848595468223

**MAPE calculated the last week**

In [19]:
mean_absolute_percentage_error(viz_df.y, viz_df.yhat)

0.034348600552228985

**Prediction starting from 9.6 until 22.6**

In [20]:
future_dates = pd.DataFrame({
    'ds': pd.date_range(start='2025-06-09', end='2025-06-22')
})



last_gold_price = gold_df['y'].iloc[-1]




forecast = model.predict(future_dates)

predictions = forecast[['ds', 'yhat']].rename(columns={
    'ds': 'date',
    'yhat': 'prediction'
})

predictions['prediction']=predictions['prediction'].round(2)
predictions['date'] = predictions['date'].dt.strftime('%Y-%m-%d')



predictions=predictions.drop([5,6,12,13])
predictions['Actual']=[3326.89,3328.18,3346.87,3388.35,3432.00,3383.79,3385.58,3366.77,3371.25,3365.18]
predictions

,date,prediction,Actual
0,2025-06-09,3128.36,3326.89
1,2025-06-10,3123.45,3328.18
2,2025-06-11,3115.02,3346.87
3,2025-06-12,3110.54,3388.35
4,2025-06-13,3100.35,3432.00
7,2025-06-16,3073.11,3383.79
8,2025-06-17,3073.28,3385.58
9,2025-06-18,3071.17,3366.77
10,2025-06-19,3074.13,3371.25
11,2025-06-20,3072.32,3365.18


**RMSE calculated starting from 9.6 until 22.6**

In [21]:
print(root_mean_squared_error(predictions.Actual,predictions.prediction))

278.8786584340938
